In [27]:
from futuquant import *
import pandas as pd
import matplotlib
import numpy as np
import talib
%matplotlib qt

In [21]:
quote_ctx = OpenQuoteContext(host='0.0.0.0', port=11111)
HK_fututre = quote_ctx.get_history_kline('HK_FUTURE.999010', '2017-11-22', '2018-11-27', ktype=KLType.K_1M)
HK_fututre = pd.DataFrame(list(HK_fututre)[1])
quote_ctx.close()

In [22]:
HK_fututre['time_key'] = (pd.to_datetime(HK_fututre['time_key'])) # Convert the time_key into datetime format
HK_fututre = HK_fututre.set_index(HK_fututre['time_key']) # Set trading hour as the index 
HK_fututre = HK_fututre.drop(['pe_ratio','turnover_rate'], axis= 1 )
normal_trading_hour = HK_fututre.between_time('09:30','16:00') #Select 09:30 - 16:00 only
#normal_trading_hour = normal_trading_hour.drop(['time_key'], axis= 1 ) #drop time_key column


In [23]:
moving_average_100 = pd.Series(talib.SMA(normal_trading_hour['close'], timeperiod=100)).values
normal_trading_hour = normal_trading_hour.assign(SMA_100 = moving_average_100)

In [24]:
normal_trading_hour['100_diff'] = normal_trading_hour['SMA_100'] - normal_trading_hour['close']

In [25]:
upper, middle, lower = talib.BBANDS(normal_trading_hour['close'], matype=talib.MA_Type.SMA, timeperiod=20)

upper = pd.Series(upper).values
normal_trading_hour = normal_trading_hour.assign(bb_upper = upper)

middle = pd.Series(middle).values
normal_trading_hour = normal_trading_hour.assign(bb_middle = middle)

lower = pd.Series(lower).values
normal_trading_hour = normal_trading_hour.assign(bb_lower = lower)

In [26]:
normal_trading_hour.loc['2018-04-27 13:58:00':'2018-04-27 13:59:00']

,code,time_key,open,close,high,low,volume,turnover,change_rate,last_close,SMA_100,100_diff,bb_upper,bb_middle,bb_lower
time_key,,,,,,,,,,,,,,,
2018-04-27 13:58:00,HK_FUTURE.999010,2018-04-27 13:58:00,30131.0,30134.0,30134.0,30131.0,5,150658.0,0.016595,30129.0,30105.23,-28.77,30139.476362,30134.20,30128.923638
2018-04-27 13:59:00,HK_FUTURE.999010,2018-04-27 13:59:00,30130.0,30134.0,30137.0,30130.0,4,120531.0,0.000000,30134.0,30105.50,-28.50,30139.167617,30134.05,30128.932383


In [36]:
order_book = []
invertory = 0
PL = 0
cut_loss_point = 1000000

for index, row in normal_trading_hour.iterrows():
    if invertory == 0: # looking for chance
        if (row['bb_upper'] - row['bb_lower'] > 100):
            if (row['100_diff'] > 0): # waiting for up trend change
                if -5 <(row['close'] - row['bb_lower']) < 5: #long  @ lower
                    print ('%s   Long  @ %d'%(str(index),int(row['close'])))              
                    order_book.append([index, -int(row['close'])]) ##
                    invertory += 1
                    continue
                if -5 <(row['close'] - row['bb_middle']) < 5: #long  @ middle
                    print ('%s   Long  @ %d'%(str(index),int(row['close'])))              
                    order_book.append([index, -int(row['close'])]) ##
                    invertory += 1
                    continue

            else:  # waiting for down trend change
                if -5 <(row['bb_middle'] - row['close']) < 5: #short  @ middle
                    print ('%s   Short  @ %d'%(str(index),int(row['close'])))
                    order_book.append([index, +int(row['close'])]) ##
                    invertory -= 1
                    continue

                if -5 <(row['bb_upper'] - row['close']) < 5: #short  @ upper
                    print ('%s   Short  @ %d'%(str(index),int(row['close'])))
                    order_book.append([index, +int(row['close'])]) ##
                    invertory -= 1
                    continue
    
    elif invertory == 1: #having long position, finding cover
        P_L = int(row['close']) + order_book[-1][1]
        
#         if P_L <= -cut_loss_point: #cutting loss
#             order_book.append([index, +int(row['close'])]) ##
#             invertory -= 1
#             PL += P_L
#             continue
        
        if -5 <(row['bb_upper'] - row['close']) < 5: #cover @ upper
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, +int(row['close'])]) ##
            invertory -= 1
            PL += P_L
            continue
            
        if -5 <(row['bb_middle'] - row['close']) < 5: #cover @ middle
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, +int(row['close'])]) ##
            invertory -= 1
            PL += P_L
            continue
            
            
    elif invertory == -1: #having short call, finding cover
        P_L = order_book[-1][1] - int(row['close'])
        
#         if P_L <= -cut_loss_point: #cutting loss
#             order_book.append([index, -int(row['close'])]) ##
#             invertory += 1
#             PL += P_L
#             continue
            
        if -5 <(row['close'] - row['bb_middle']) < 5: #cover @ middle
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, -int(row['close'])]) ##
            invertory += 1
            PL += P_L
            continue
            
            
        if -5 <(row['close'] - row['bb_lower']) < 5: #cover @ lower
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, -int(row['close'])]) ##
            invertory += 1
            PL += P_L
            continue
        
    #cover the last invertory
#     if index == normal_trading_hour.iloc[-1:].index:
#         if invertory > 0 :
#             order_book.append([index, +int(row['close'])]) ##
#             invertory -= 1
        
#         if invertory < 0 :
#             order_book.append([index, -int(row['close'])]) ##
#             invertory += 1

2018-04-03 10:14:00   Short  @ 29820
2018-04-03 10:28:00   cover  @ 29796
Profit and Loss = 24
2018-04-03 10:45:00   Short  @ 29921
2018-04-03 10:56:00   cover  @ 29886
Profit and Loss = 35
2018-04-03 14:17:00   Short  @ 29994
2018-04-03 14:31:00   cover  @ 30001
Profit and Loss = -7
2018-04-03 14:44:00   Short  @ 30071
2018-04-03 15:25:00   cover  @ 30248
Profit and Loss = -177
2018-04-04 09:34:00   Short  @ 30280
2018-04-04 10:01:00   cover  @ 30197
Profit and Loss = 83
2018-04-04 10:03:00   Long  @ 30200
2018-04-04 10:25:00   cover  @ 30204
Profit and Loss = 4
2018-04-04 10:28:00   Long  @ 30139
2018-04-04 10:35:00   cover  @ 30174
Profit and Loss = 35
2018-04-04 13:03:00   Short  @ 30157
2018-04-04 13:11:00   cover  @ 30125
Profit and Loss = 32
2018-04-04 14:20:00   Long  @ 29943
2018-04-04 14:54:00   cover  @ 29839
Profit and Loss = -104
2018-04-04 15:09:00   Long  @ 29743
2018-04-04 15:33:00   cover  @ 29685
Profit and Loss = -58
2018-04-04 15:50:00   Long  @ 29617
2018-04-06 09:

2018-04-23 09:31:00   cover  @ 30381
Profit and Loss = -7
2018-04-23 09:34:00   Long  @ 30356
2018-04-23 09:40:00   cover  @ 30419
Profit and Loss = 63
2018-04-23 09:45:00   Short  @ 30490
2018-04-23 10:07:00   cover  @ 30246
Profit and Loss = 244
2018-04-23 10:10:00   Long  @ 30201
2018-04-23 10:19:00   cover  @ 30233
Profit and Loss = 32
2018-04-23 10:29:00   Long  @ 30206
2018-04-23 10:45:00   cover  @ 30205
Profit and Loss = -1
2018-04-23 10:53:00   Short  @ 30294
2018-04-23 11:13:00   cover  @ 30358
Profit and Loss = -64
2018-04-23 11:32:00   Short  @ 30347
2018-04-23 11:34:00   cover  @ 30347
Profit and Loss = 0
2018-04-23 15:26:00   Long  @ 30273
2018-04-23 15:44:00   cover  @ 30243
Profit and Loss = -30
2018-04-24 09:55:00   Short  @ 30438
2018-04-24 10:30:00   cover  @ 30577
Profit and Loss = -139
2018-04-24 11:11:00   Short  @ 30664
2018-04-24 11:32:00   cover  @ 30632
Profit and Loss = 32
2018-04-24 15:20:00   Short  @ 30623
2018-04-24 15:32:00   cover  @ 30605
Profit and Lo

2018-06-08 10:01:00   Long  @ 31241
2018-06-08 10:17:00   cover  @ 31235
Profit and Loss = -6
2018-06-08 11:13:00   Long  @ 31106
2018-06-08 11:24:00   cover  @ 31106
Profit and Loss = 0
2018-06-08 14:15:00   Long  @ 30958
2018-06-08 14:30:00   cover  @ 30967
Profit and Loss = 9
2018-06-08 15:05:00   Long  @ 30859
2018-06-08 15:13:00   cover  @ 30893
Profit and Loss = 34
2018-06-08 15:14:00   Long  @ 30889
2018-06-08 15:15:00   cover  @ 30885
Profit and Loss = -4
2018-06-11 09:52:00   Short  @ 30941
2018-06-11 10:11:00   cover  @ 31006
Profit and Loss = -65
2018-06-12 09:49:00   Short  @ 31136
2018-06-12 10:03:00   cover  @ 31157
Profit and Loss = -21
2018-06-12 10:14:00   Long  @ 30981
2018-06-12 10:25:00   cover  @ 30994
Profit and Loss = 13
2018-06-12 10:38:00   Short  @ 31068
2018-06-12 10:51:00   cover  @ 31070
Profit and Loss = -2
2018-06-12 13:11:00   Short  @ 31129
2018-06-12 13:12:00   cover  @ 31128
Profit and Loss = 1
2018-06-13 09:50:00   Long  @ 30944
2018-06-13 09:55:00  

2018-07-05 09:52:00   Long  @ 28060
2018-07-05 09:57:00   cover  @ 28146
Profit and Loss = 86
2018-07-05 09:58:00   Short  @ 28142
2018-07-05 10:03:00   cover  @ 28122
Profit and Loss = 20
2018-07-05 10:06:00   Short  @ 28118
2018-07-05 10:14:00   cover  @ 28112
Profit and Loss = 6
2018-07-05 10:19:00   Short  @ 28099
2018-07-05 10:23:00   cover  @ 28097
Profit and Loss = 2
2018-07-05 10:38:00   Short  @ 28203
2018-07-05 10:48:00   cover  @ 28176
Profit and Loss = 27
2018-07-05 11:03:00   Long  @ 28108
2018-07-05 11:41:00   cover  @ 27962
Profit and Loss = -146
2018-07-05 13:14:00   Long  @ 27734
2018-07-05 13:19:00   cover  @ 27820
Profit and Loss = 86
2018-07-05 14:06:00   Short  @ 27886
2018-07-05 14:10:00   cover  @ 27895
Profit and Loss = -9
2018-07-05 15:21:00   Short  @ 28114
2018-07-05 15:29:00   cover  @ 28060
Profit and Loss = 54
2018-07-05 16:00:00   Short  @ 28042
2018-07-06 09:53:00   cover  @ 28175
Profit and Loss = -133
2018-07-06 10:04:00   Long  @ 28014
2018-07-06 10:1

2018-08-09 10:57:00   cover  @ 28497
Profit and Loss = -132
2018-08-09 13:14:00   Short  @ 28632
2018-08-09 13:21:00   cover  @ 28611
Profit and Loss = 21
2018-08-09 14:56:00   Long  @ 28500
2018-08-09 15:08:00   cover  @ 28514
Profit and Loss = 14
2018-08-10 09:55:00   Short  @ 28556
2018-08-10 10:07:00   cover  @ 28535
Profit and Loss = 21
2018-08-10 10:36:00   Long  @ 28404
2018-08-10 10:53:00   cover  @ 28438
Profit and Loss = 34
2018-08-10 11:10:00   Long  @ 28377
2018-08-10 11:19:00   cover  @ 28394
Profit and Loss = 17
2018-08-10 13:11:00   Long  @ 28321
2018-08-10 13:45:00   cover  @ 28210
Profit and Loss = -111
2018-08-10 15:51:00   Long  @ 28204
2018-08-10 16:00:00   cover  @ 28224
Profit and Loss = 20
2018-08-13 09:59:00   Long  @ 27756
2018-08-13 10:10:00   cover  @ 27770
Profit and Loss = 14
2018-08-13 13:12:00   Short  @ 27788
2018-08-13 13:31:00   cover  @ 27836
Profit and Loss = -48
2018-08-13 14:36:00   Short  @ 27964
2018-08-13 14:44:00   cover  @ 27912
Profit and Los

2018-09-14 09:47:00   Short  @ 27078
2018-09-14 09:50:00   cover  @ 27084
Profit and Loss = -6
2018-09-14 09:51:00   Short  @ 27077
2018-09-14 09:57:00   cover  @ 27083
Profit and Loss = -6
2018-09-14 10:36:00   Short  @ 27212
2018-09-14 10:38:00   cover  @ 27204
Profit and Loss = 8
2018-09-14 15:06:00   Long  @ 27251
2018-09-14 15:13:00   cover  @ 27275
Profit and Loss = 24
2018-09-17 09:59:00   Long  @ 26861
2018-09-17 10:16:00   cover  @ 26803
Profit and Loss = -58
2018-09-17 10:17:00   Long  @ 26797
2018-09-17 10:19:00   cover  @ 26791
Profit and Loss = -6
2018-09-17 14:01:00   Short  @ 26892
2018-09-17 14:20:00   cover  @ 26899
Profit and Loss = -7
2018-09-17 15:45:00   Short  @ 26975
2018-09-17 15:52:00   cover  @ 26942
Profit and Loss = 33
2018-09-18 09:44:00   Long  @ 26776
2018-09-18 10:13:00   cover  @ 26757
Profit and Loss = -19
2018-09-18 10:24:00   Long  @ 26717
2018-09-18 10:40:00   cover  @ 26706
Profit and Loss = -11
2018-09-18 11:11:00   Short  @ 26731
2018-09-18 11:12

2018-10-24 10:19:00   Short  @ 25344
2018-10-24 10:22:00   cover  @ 25281
Profit and Loss = 63
2018-10-24 10:26:00   Short  @ 25374
2018-10-24 10:32:00   cover  @ 25314
Profit and Loss = 60
2018-10-24 10:36:00   Long  @ 25246
2018-10-24 10:50:00   cover  @ 25251
Profit and Loss = 5
2018-10-24 11:02:00   Short  @ 25389
2018-10-24 11:52:00   cover  @ 25587
Profit and Loss = -198
2018-10-24 13:18:00   Short  @ 25478
2018-10-24 13:26:00   cover  @ 25378
Profit and Loss = 100
2018-10-24 13:39:00   Long  @ 25387
2018-10-24 13:44:00   cover  @ 25433
Profit and Loss = 46
2018-10-24 14:48:00   Long  @ 25282
2018-10-24 14:49:00   cover  @ 25278
Profit and Loss = -4
2018-10-24 14:50:00   Long  @ 25275
2018-10-24 14:52:00   cover  @ 25270
Profit and Loss = -5
2018-10-25 10:00:00   Long  @ 24637
2018-10-25 10:09:00   cover  @ 24677
Profit and Loss = 40
2018-10-25 10:32:00   Long  @ 24747
2018-10-25 10:33:00   cover  @ 24742
Profit and Loss = -5
2018-10-25 10:37:00   Long  @ 24743
2018-10-25 10:48:0

In [35]:
print ("Profit and Loss (in point): %d" % PL)
print ("Profit and Loss (HKD): %d" % (PL*50))
print ("Trade times: %d" % int(len(order_book)/2))

Profit and Loss (in point): 2012
Profit and Loss (HKD): 100600
Trade times: 828


In [33]:
len(order_book)/2

953.0

In [ ]:
order_book_2 = []
invertory_2 = 0
for index, row in normal_trading_hour.iloc[99:].iterrows():
    
    if invertory_2 == 0: # looking for chance
        if (row['100_diff'] > 0): # waiting for up trend change
            if row['signal'] == 'long_call': #buy chance found
                order_book_2.append([index, -int(row['close'])]) ##
                invertory_2 += 1
        else:
            if row['signal'] == 'short_call': #short chance found
                order_book_2.append([index, +int(row['close'])]) ##
                invertory_2 -= 1
                
    
    
    if invertory_2 == 1: #having long call, finding cover
        if row['signal'] == 'short_call': #cover
            order_book_2.append([index, +int(row['close'])]) ##
            invertory_2 -= 1
            
            
    if invertory_2 == -1: #having short call, finding cover
        if row['signal'] == 'long_call': #cover
            order_book_2.append([index, -int(row['close'])]) ##
            invertory_2 += 1
        
        
        
    #cover the last invertory
    if index == normal_trading_hour.iloc[-1:].index:
        if invertory_2 > 0 :
            order_book_2.append([index, +int(row['close'])]) ##
            invertory_2 -= 1
        
        if invertory_2 < 0 :
            order_book_2.append([index, -int(row['close'])]) ##
            invertory_2 += 1

In [ ]:
class Formatter(Formatter):
    def __init__(self, dates, fmt='%Y-%m-%d %H:%M:%S'):
        self.dates = dates
        self.fmt = fmt

    def __call__(self, x, pos=0):
        'Return the label for time x at position pos'
        
        ind = int(np.round(x))
        
        if ind >= len(self.dates) or ind < 0:
            return ''
        return self.dates[ind].strftime(self.fmt)



In [ ]:
tmp.head(2)

In [ ]:
fig, axarr = plt.subplots(2, sharex=True)

axarr[0].xaxis.set_major_formatter(formatter)

axarr[0].plot(np.arange(len(tmp)), tmp['close'], '-')
axarr[0].plot(np.arange(len(tmp)), tmp['bb_middle'], '-')
axarr[0].plot(np.arange(len(tmp)), tmp['bb_upper'], '-')
axarr[0].plot(np.arange(len(tmp)), tmp['bb_lower'], '-')



axarr[1].plot(np.arange(len(tmp)), tmp['100_diff'], '-')

fig.subplots_adjust(left=0.1, right=0.95, top=1, bottom=0.15)


plt.show()

In [ ]:
class trading_with_sma100:
    def __init__(self,dt):
        self.dt = dt
        dt['time_key'] = (pd.to_datetime(dt['time_key'])) # Convert the time_key into datetime format
        dt = dt.set_index(dt['time_key']) # Set trading hour as the index 
        dt = dt.drop(['pe_ratio','turnover_rate'], axis= 1 )
        normal_trading_hour = dt.between_time('09:30','16:00') #Select 09:30 - 16:00 only
        #normal_trading_hour = normal_trading_hour.drop(['time_key'], axis= 1 ) #drop time_key column
        
        moving_average_100 = pd.Series(talib.SMA(normal_trading_hour['close'], timeperiod=100)).values #add sma 100 into dt
        normal_trading_hour = normal_trading_hour.assign(SMA_100 = moving_average_100)
        normal_trading_hour['100_diff'] = normal_trading_hour['SMA_100'] - normal_trading_hour['close']
        
        
        upper, middle, lower = talib.BBANDS(normal_trading_hour['close'], matype=talib.MA_Type.SMA, timeperiod=20) #add bollinger bands

        upper = pd.Series(upper).values
        normal_trading_hour = normal_trading_hour.assign(bb_upper = upper)

        middle = pd.Series(middle).values
        normal_trading_hour = normal_trading_hour.assign(bb_middle = middle)

        lower = pd.Series(lower).values
        normal_trading_hour = normal_trading_hour.assign(bb_lower = lower)
        
        self.dt = normal_trading_hour
        
    def buy_sell_single(self):
        # Long Call
        normal_trading_hour.loc[(normal_trading_hour['close'] > normal_trading_hour['bb_lower']) & (normal_trading_hour['close'] - normal_trading_hour['bb_lower'] < 10), 'signal'] = 'long_call'
        normal_trading_hour.loc[(normal_trading_hour['close'] > normal_trading_hour['bb_middle']) & (normal_trading_hour['close'] - normal_trading_hour['bb_middle'] < 10), 'signal'] = 'long_call'

        #Short Call
        normal_trading_hour.loc[(normal_trading_hour['close'] < normal_trading_hour['bb_upper']) & (normal_trading_hour['bb_lower'] - normal_trading_hour['close'] < 10), 'signal'] = 'short_call'
        normal_trading_hour.loc[(normal_trading_hour['close'] < normal_trading_hour['bb_middle']) & (normal_trading_hour['bb_middle'] - normal_trading_hour['close'] < 10), 'signal'] = 'short_call'
        
        
    def order_book(self):
        
        book = dict()
        

In [17]:
from helper.back_testing import back_testing

In [18]:
HK_future_backtesting = back_testing()

In [ ]:
for index, row in normal_trading_hour.iterrows():
    if invertory == 0: # looking for chance
        if (row['bb_upper'] - row['bb_lower'] > 100):
            if (row['100_diff'] > 0): # waiting for up trend change
                if -5 <(row['close'] - row['bb_lower']) < 5: #long  @ lower
                    print ('%s   Long  @ %d'%(str(index),int(row['close'])))              
                    order_book.append([index, -int(row['close'])]) ##
                    invertory += 1
                    continue
                if -5 <(row['close'] - row['bb_middle']) < 5: #long  @ middle
                    print ('%s   Long  @ %d'%(str(index),int(row['close'])))              
                    order_book.append([index, -int(row['close'])]) ##
                    invertory += 1
                    continue

            else:  # waiting for down trend change
                if -5 <(row['bb_middle'] - row['close']) < 5: #short  @ middle
                    print ('%s   Short  @ %d'%(str(index),int(row['close'])))
                    order_book.append([index, +int(row['close'])]) ##
                    invertory -= 1
                    continue

                if -5 <(row['bb_upper'] - row['close']) < 5: #short  @ upper
                    print ('%s   Short  @ %d'%(str(index),int(row['close'])))
                    order_book.append([index, +int(row['close'])]) ##
                    invertory -= 1
                    continue
    
    elif invertory == 1: #having long position, finding cover
        P_L = int(row['close']) + order_book[-1][1]
        
        if P_L <= -cut_loss_point: #cutting loss
            order_book.append([index, +int(row['close'])]) ##
            invertory -= 1
            PL += P_L
            continue
        
        if 0 <(row['bb_upper'] - row['close']) < 10: #cover @ upper
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, +int(row['close'])]) ##
            invertory -= 1
            PL += P_L
            continue
            
        if 0 <(row['bb_middle'] - row['close']) < 10: #cover @ middle
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, +int(row['close'])]) ##
            invertory -= 1
            PL += P_L
            continue
            
            
    elif invertory == -1: #having short call, finding cover
        P_L = order_book[-1][1] - int(row['close'])
        
        if P_L <= -cut_loss_point: #cutting loss
            order_book.append([index, -int(row['close'])]) ##
            invertory += 1
            PL += P_L
            continue
            
        if 0 <(row['close'] - row['bb_middle']) < 10: #cover @ middle
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, -int(row['close'])]) ##
            invertory += 1
            PL += P_L
            continue
            
            
        if 0 <(row['close'] - row['bb_lower']) < 10: #cover @ lower
            print ('%s   cover  @ %d'%(str(index),int(row['close'])))
            
            print ('Profit and Loss = %d'%(P_L))
            
            order_book.append([index, -int(row['close'])]) ##
            invertory += 1
            PL += P_L
            continue

In [39]:
inventory = dict()
inventory['long'] = []
inventory['Short'] = []

In [40]:
len(inventory)

2

In [42]:
len(inventory['long']) + len(inventory['Short'])

0